In [9]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.gconvGRU_simple import GConvGRUModel
from memory_capacity_utils import gen_lag_data_embeddings, get_mem_cap_from_model

In [10]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')


running on GPU


In [11]:
dataset = np.load('datasets/oasis_data.npy')
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)
dataset = np.delete(dataset,88,axis=0)

# dataset = np.load("datasets/multivariate_simulation_data_2.npy")
# dataset = torch.from_numpy(dataset).squeeze()
# dataset = dataset.type(torch.FloatTensor)

# Load the dataset

In [12]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=3, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=25, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.001, help="Learning rate")
    parser.add_argument('--memcap_coef', type=float, default=0, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [13]:
args = get_args()

# Set up the network architecture and train-validate

**Train-validate-functions**

In [14]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

embeddings = np.load('datasets/gutenberg_embeddings.npy')
# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data_embeddings(embeddings, 1000, args.max_lag, 41)
X_test_res_np, y_test_res_np = gen_lag_data_embeddings(embeddings, 500, args.max_lag, 42)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

In [15]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/baseline_gconvGRU_language_2/test_tp_losses' already exists.
Director

In [17]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [27]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_oasis_language.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    # Create model instance
    model = GConvGRUModel(device=device, input_weights=input_weights, input_scaling=1e-1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1): 
                pred = model(data[t])
                real = data[t + 1]
        
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            
            total_loss = mae_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                            X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/3]-----------------------------------------
Epoch [1/25]
[Train] MAE Loss: 0.05095640538881222, TP Loss: 3.3975850254297257
[Validate] MAE Loss Across Timepoints: [0.0768441  0.06443176]
[Validate] TP Loss Across Timepoints: [11.12688293  2.19567998]
[Validate] MAE of Mem Caps Across Timepoints: [0.57337221 0.71241179]
Epoch [2/25]
[Train] MAE Loss: 0.04335717642679811, TP Loss: 2.758782848715782
[Validate] MAE Loss Across Timepoints: [0.06538308 0.05361247]
[Validate] TP Loss Across Timepoints: [9.30393982 1.55640043]
[Validate] MAE of Mem Caps Across Timepoints: [0.52840852 0.57151708]
Epoch [3/25]
[Train] MAE Loss: 0.03859871405487259, TP Loss: 2.330310787508885
[Validate] MAE Loss Across Timepoints: [0.0616601  0.04969955]
[Validate] TP Loss Across Timepoints: [8.82451782 1.39674555]
[Validate] MAE of Mem Caps Across Timepoints: [0.52233398 0.65552041]
Epoch [4/25]
[Train] MAE Loss: 0.035921758382270734, TP Loss: 2.16670040016373
[Validat

[Validate] MAE Loss Across Timepoints: [0.0597888  0.05555879]
[Validate] TP Loss Across Timepoints: [8.76103414 3.12146428]
[Validate] MAE of Mem Caps Across Timepoints: [0.49203468 0.71122593]
Epoch [5/25]
[Train] MAE Loss: 0.034523792254428066, TP Loss: 1.7756375884016355
[Validate] MAE Loss Across Timepoints: [0.05853408 0.05392247]
[Validate] TP Loss Across Timepoints: [8.49249471 2.90131226]
[Validate] MAE of Mem Caps Across Timepoints: [0.54321201 0.72200769]
Epoch [6/25]
[Train] MAE Loss: 0.034175263221065205, TP Loss: 1.7219595129291216
[Validate] MAE Loss Across Timepoints: [0.05666762 0.05287096]
[Validate] TP Loss Across Timepoints: [8.13605143 2.75041224]
[Validate] MAE of Mem Caps Across Timepoints: [0.6295787 0.7810929]
Epoch [7/25]
[Train] MAE Loss: 0.03275969338913758, TP Loss: 1.6154164418578147
[Validate] MAE Loss Across Timepoints: [0.05573877 0.05190353]
[Validate] TP Loss Across Timepoints: [8.10311534 2.66389109]
[Validate] MAE of Mem Caps Across Timepoints: [0.4

[Train] MAE Loss: 0.030808371119201185, TP Loss: 1.6311018099387486
[Validate] MAE Loss Across Timepoints: [0.04156931 0.04446583]
[Validate] TP Loss Across Timepoints: [1.60966551 1.68739915]
[Validate] MAE of Mem Caps Across Timepoints: [0.97298003 0.8486171 ]
Epoch [9/25]
[Train] MAE Loss: 0.030578241466234127, TP Loss: 1.6678011655807494
[Validate] MAE Loss Across Timepoints: [0.04164179 0.04457339]
[Validate] TP Loss Across Timepoints: [1.62054157 1.70539796]
[Validate] MAE of Mem Caps Across Timepoints: [0.93106833 0.8805858 ]
Epoch [10/25]
[Train] MAE Loss: 0.03058295880133907, TP Loss: 1.6487637432912985
[Validate] MAE Loss Across Timepoints: [0.041328   0.04415325]
[Validate] TP Loss Across Timepoints: [1.55399346 1.61633801]
[Validate] MAE of Mem Caps Across Timepoints: [0.91376273 0.81144308]
Epoch [11/25]
[Train] MAE Loss: 0.030360520537942647, TP Loss: 1.6352877487738928
[Validate] MAE Loss Across Timepoints: [0.04135643 0.04419596]
[Validate] TP Loss Across Timepoints: [1